In [20]:
import pandas as pd
import numpy as np

# Importing data

In [21]:
SnpSiftData=pd.read_csv(r'data/snpsiftdata.tab',delimiter='\t',encoding='UTF-8')
chromosome_name=pd.read_csv(r'data/chromosome_name_mapping.tab',delimiter='\t',encoding='UTF-8')
Genotype_Data=pd.read_csv(r'data/genotype_data.tab',delimiter='\t',encoding='UTF-8')
passport=pd.read_csv(r'data/passport.tab',delimiter='\t',encoding='UTF-8')

In [22]:
passport.replace(np.NaN,'NA',inplace=True)

In [23]:
Genotype_Data_1=pd.melt(Genotype_Data,id_vars=['CHROM','POS'],var_name='Sample_ID',value_name='GT')

# Adding a unique column based on Chromosome name and Position in both the Genotype_Data_1 file and SnpSiftData file

In [24]:
Genotype_Data_1['Chrom_Pos']=Genotype_Data_1['CHROM'].astype(str).str.cat(Genotype_Data_1['POS'].astype(str),sep='_')
SnpSiftData['Chrom_Pos']=SnpSiftData['CHROM'].astype(str).str.cat(SnpSiftData['POS'].astype(str),sep='_')

# Adding the information {Variety, Generation} from the passport file into Genotype_Data_1 file.

In [25]:
Genotype_Data_1.insert(4,'Variety',Genotype_Data_1['Sample_ID'].map(passport.set_index('Sample-ID')['Variety']))
Genotype_Data_1.insert(5,'Generation',Genotype_Data_1['Sample_ID'].map(passport.set_index('Sample-ID')['Generation']))
Genotype_Data_1.insert(5,'Treatment',Genotype_Data_1['Sample_ID'].map(passport.set_index('Sample-ID')['Treatment']))
Genotype_Data_1.insert(5,'Dose',Genotype_Data_1['Sample_ID'].map(passport.set_index('Sample-ID')['Dose']))

# Adding the chromosome names from the chromosome_name file.(Note the names will the same at first but if the user updates them they will be updated)

In [26]:
SnpSiftData.insert(5,'chrome_name',SnpSiftData['CHROM'].map(chromosome_name.set_index('Contig')['Chromosome']))
SnpSiftData.rename(columns={'CHROM': 'CONTIG', 'chrome_name':'CHROM'}, inplace=True)
SnpSiftData.replace(np.NaN, 'NA', inplace=True)

In [27]:
Genotype_Data_1.insert(1, 'chrome_name', Genotype_Data_1['CHROM'].map(chromosome_name.set_index('Contig')['Chromosome']))
Genotype_Data_1.rename(columns={'CHROM': 'CONTIG', 'chrome_name':'CHROM'}, inplace=True)
Genotype_Data_1.replace(np.NaN,'NA',inplace=True)

# Merge the Genotype and SNP dataframes

In [28]:
master = pd.merge(SnpSiftData, Genotype_Data_1)

# Selecting and Ordering Columns

In [29]:
columns = [
    'ANN[*].GENE',
    'CHROM',
    'CONTIG',
    'POS',
    'Sample_ID',
    'Variety',
    'Generation',
    'Treatment',
    'Dose',
    'GT',
    'REF',
    'ALT',
    'TYPE',
    'ANN[*].IMPACT',
    'ANN[*].EFFECT',
    'ANN[*].FEATURE',
    'ANN[*].BIOTYPE',
    'ANN[*].DISTANCE',
    'ID'   
]

# Creating Final Dataframe

In [30]:
master_final = master[columns]

# Exporting data

In [31]:
master_final.to_csv('master_data.csv', index=False, header=True)